In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/kaggle/notebook

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/kaggle/notebook'
/content/drive/MyDrive/kaggle/notebook


In [ ]:
%%capture
!pip install transformers==4.20.1
!pip install tokenizers==0.12.1
!pip install sentencepiece==0.1.97

In [ ]:
import os
import gc
import time
import math
import datetime

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


import transformers
import tokenizers
from transformers import AutoModel, AutoTokenizer
from transformers import get_cosine_schedule_with_warmup

os.system('pip install iterative-stratification==0.1.7')
os.environ['TOKENIZERS_PARALLELISM']='true'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CFG:
    model_name  = "microsoft/deberta-v3-large"
    
    base        = "../input/feedback-prize-english-language-learning/"
    train       = base + "train.csv"
    test        = base + "test.csv"
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    num_targets = 6
    
    # Backbone
    max_length  = 512
    hidden_dims = 1024
    output_hidden_states = True
    pool = 'attention'
    
    # loss
    loss = 'L1smooth'
    apex = True
    max_norm = 300
    
    # optimizer
    default_lr = 2e-5
    head_lr    = 3e-4
    adam_eps   = 1e-5
    
    # scheduler
    scheduler = 'linear'
    num_warmup_steps = 0.333333
    
    # CV
    n_folds = 5
    seed   = 42
    epochs = 5
    
    # Loader
    batch_size  = 2
    num_workers = 2
    print_freq = 20

    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [ ]:
if len(CFG.model_name.split("/")) == 2:
   CFG.identifier = f"{CFG.str_now}-{CFG.model_name.split('/')[1]}"
else:
   CFG.identifier = f"{CFG.str_now}-{CFG.model_name}"

CFG.OUTPUT_DIR = f'./{CFG.identifier}/'
os.makedirs(CFG.OUTPUT_DIR, exist_ok = True)

In [ ]:
df_train = pd.read_csv(CFG.train)

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
fold = MultilabelStratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
df   = df_train.copy()
y    = pd.get_dummies(data = df[CFG.target_cols], columns = CFG.target_cols)

for idx, (train_idx, valid_idx) in enumerate(fold.split(df_train, y)):
    df_train.loc[valid_idx, 'fold'] = idx

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
tokenizer.add_tokens(["\n"], special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


1

In [ ]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
             
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f'{int(m)}m {int(s)}s'
        
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    
    return f'{str(asMinutes(s))} (remain {str(asMinutes(rs))})'
        
def prepare_inputs(texts):
    inputs = []
    for text in texts:
        tokens = tokenizer.encode_plus(text,
                                       add_special_tokens = True,
                                       max_length = CFG.max_length,
                                       pad_to_max_length = True,
                                       truncation = True,
                                       return_attention_mask = True
                                       )
        
        for key, vals in tokens.items():
            tokens[key] = torch.tensor(vals, dtype = torch.long)
        inputs.append(tokens)
    
    return inputs

class MeanPooling(nn.Module):
      def __init__(self):
          super(MeanPooling, self).__init__()

      def forward(self, inputs, attention_masks):
          #print("inputs.shape: ", inputs.shape, " masks.shape: ", attention_masks.shape)
          attention_masks = attention_masks.unsqueeze(-1).float()
          inputs   = torch.sum(inputs*attention_masks, dim = 1)
          num_masks = torch.sum(attention_masks == 1.0, dim = 1)
          torch.clamp(num_masks, min = 1e-9)
          inputs   = (inputs / num_masks)

          return inputs

class AttentionPooling(nn.Module):
      def __init__(self, hidden_size):
          super(AttentionPooling, self).__init__()
          self.attention = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                         nn.GELU(),
                                         nn.Linear(hidden_size, 1)
                                         )

      def forward(self, hidden_state, attention_mask):
          weights = self.attention(hidden_state)
          weights[attention_mask == 0] = float("-inf")
          weights = torch.softmax(weights, dim = 1)
          context = torch.sum(hidden_state * weights, dim = 1)

          return context

def MCRMSE(labels, preds):
    scores = []
    num_targets = labels.shape[1]
    
    for i in range(num_targets):
        pred  = preds[:,i]
        label = labels[:,i]
        score = mean_squared_error(label, pred, squared = False)
        scores.append(score)
    
    mcrmse = np.mean(scores)
    
    return mcrmse, scores 

def get_score(labels, preds):
    return MCRMSE(labels, preds)
        

In [ ]:
class FB3Dataset(Dataset):
    def __init__(self, df_train):
        self.inputs = prepare_inputs(df_train['full_text'])
        self.labels = torch.tensor(df_train[CFG.target_cols].values, dtype = torch.float)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

In [ ]:
class FB3Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.model      = AutoModel.from_pretrained(cfg.model_name, 
                                                    output_hidden_states=cfg.output_hidden_states)
        self.cfg        = cfg

        if self.cfg.pool == 'mean':
          self.pool       = MeanPooling()
          self.classifier = nn.Linear(cfg.hidden_dims, cfg.num_targets)
        elif self.cfg.pool == 'attention':
          hidden_size = cfg.hidden_dims * 4
          self.lstm = nn.LSTM(input_size = hidden_size, 
                              hidden_size = hidden_size // 2,
                              batch_first = True,
                              bidirectional = True)
          self.pool = AttentionPooling(hidden_size)
          #self.pool       = MeanPooling()
        
          self.classifier = nn.Linear(hidden_size, cfg.num_targets)
    
    def feature(self, inputs):
        outputs = self.model(**inputs)

        if self.cfg.pool == 'mean':
           feature = self.pool(outputs.last_hidden_state, inputs['attention_mask'])
        elif self.cfg.pool == 'attention':
           hidden_states = outputs.hidden_states
           #print("hidden_states.shape: ", len(hidden_states))
           stacked_outputs = torch.cat([hidden_states[-i-1] for i in range(4)], dim = -1)
           #print("stacked_outputs.shape: ", stacked_outputs.shape)
           feature, _ = self.lstm(stacked_outputs)
           #print("feature.shape: ", feature.shape)
           feature = self.pool(feature, inputs['attention_mask'])
           #print("last.shape: ", feature.shape)
        
        return feature
    
    def forward(self, inputs):

        feature = self.feature(inputs)
        outputs = self.classifier(feature)
        
        return outputs
    

In [ ]:
def train_fn(model, criterion, optimizer, scheduler, train_loader, epoch, cfg):
    losses = AverageMeter()
    total_steps = len(train_loader)
    start = end = time.time()
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.apex)
    
    for step, (inputs, labels) in enumerate(train_loader):
        #print("inputs.shape: ", inputs['input_ids'].shape)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        
        with torch.cuda.amp.autocast(enabled = cfg.apex):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        losses.update(loss.item(), labels.shape[0])
        
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_norm)
          
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        
        end = time.time()
        
        if (step + 1) % cfg.print_freq == 0:
            print("[{0}][{1}/{2}] "
                  "Elapsed: {remain:s} "
                  "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                  "Grad: {grad_norm} "
                  "LR: {lr:.8f} "
                  .format(epoch, step, total_steps, remain = timeSince(start, float(step+1)/total_steps),
                          loss = losses,
                          grad_norm = grad_norm,
                          lr = scheduler.get_lr()[0]
                          )
                 )
    
    return losses.avg

def valid_fn(model, criterion, valid_loader, epoch, cfg):
    losses = AverageMeter()
    predictions = []
    total_steps = len(valid_loader) - 1
    model.eval()
    start = end = time.time()
    
    for step, (inputs, labels) in enumerate(valid_loader):
        
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            preds = model(inputs)
        
        loss = criterion(preds, labels)
        losses.update(loss.item(), labels.shape[0])
        
        predictions.append(preds.detach().cpu().numpy())
        
        if (step + 1) % cfg.print_freq == 0 or step == total_steps:
            print("[{0}][{1}/{2}] "
                  "Elapsed: {remain:s} "
                  "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                  .format(epoch, step, total_steps, remain = timeSince(start, float(step+1)/total_steps),
                          loss = losses,
                          )
                 )
    
    predictions = np.concatenate(predictions, axis = 0)
    
    return losses.avg, predictions

In [ ]:
from collections import defaultdict

def train_loop(folds, fold, cfg):
    train_folds  = folds[folds['fold'] != fold]
    valid_folds  = folds[folds['fold'] == fold]
    valid_labels = valid_folds[cfg.target_cols].values
    
    train_dataset = FB3Dataset(train_folds)
    valid_dataset = FB3Dataset(valid_folds)
    
    train_loader  = DataLoader(train_dataset, 
                               batch_size  = cfg.batch_size,
                               shuffle = True,
                               pin_memory = True,
                               num_workers = cfg.num_workers)
    
    valid_loader  = DataLoader(valid_dataset, 
                               batch_size = cfg.batch_size,
                               shuffle = False,
                               pin_memory = True,
                               num_workers = cfg.num_workers)
    
    def get_optimizer(cfg, model):
        
        all_parameters = list(model.named_parameters())
        used_name_parameters = set()
        params = []
        named_params = []
        no_wd = ['word_embeddings', 'bias', 'LayerNorm.weight']

        head   = [(n, p) for n, p in model.named_parameters() if not 'model' in n]
        for n, _ in head: used_name_parameters.add(n)
  
        params.append({"params": [p for n, p in head if not any(no in n for no in no_wd)], "weight_decay": 0.01, "lr": cfg.head_lr})
        named_params.append({"params": [n for n, p in head if not any(no in n for no in no_wd)], "weight_decay": 0.01, "lr": cfg.head_lr})
        params.append({"params": [p for n, p in head if any(no in n for no in no_wd)], "weight_decay": 0.0, "lr": cfg.head_lr})
        named_params.append({"params": [n for n, p in head if any(no in n for no in no_wd)], "weight_decay": 0.0, "lr": cfg.head_lr})

        backbone = [(n, p) for n, p in model.named_parameters() if 'model' in n]
        
        groups = [
            [ [".embeddings."],                           1e-6],
            [ ["encoder.LayerNorm", "rel_embeddings"],    1e-6],
            [ ["." + str(i) + "." for i in range(0,6)],   1e-8],
            [ ["." + str(i) + "." for i in range(6,12)],  1e-7],
            [ ["." + str(i) + "." for i in range(12,23)], 1e-6],
            [ [".23."], cfg.head_lr]
        ]

        for group in groups:
            names, lr = group[0], group[1]
            print("names: ", names)
            parameters = [(n, p) for n, p in backbone if any(name in n for name in names)]
            for n, _ in parameters: used_name_parameters.add(n)

            params.append({"params": [p for n, p in parameters if not any(no in n for no in no_wd)], 'weight_decay': 0.01, "lr": lr})
            named_params.append({"params": [n for n, p in parameters if not any(no in n for no in no_wd)], 'weight_decay': 0.01, "lr": lr})
            params.append({"params": [p for n, p in parameters if any(no in n for no in no_wd)], 'weight_decay': 0.0, "lr": lr})
            named_params.append({"params": [n for n, p in parameters if any(no in n for no in no_wd)], 'weight_decay': 0.0, "lr": lr})
        

        param_dict = defaultdict()
        for param in params:
            for _param in param['params']:
                key = id(_param)
                param_dict[key] = param_dict.get(key, 0) + 1
        
        for key, val in param_dict.items():
           if val > 1: print("key: ", key, " val: ", val)
        optimizer = torch.optim.AdamW(params, eps = cfg.adam_eps)
        
        state_dict_keys = {n:p for n, p in all_parameters}.keys()
        assert(len(state_dict_keys - used_name_parameters) == 0), \
               f"Missing parameters: {str(state_dict_keys-used_name_parameters)}"

        return optimizer
    
    def get_scheduler(cfg, optimizer, num_train_steps):
        warmup_start = int(cfg.epochs * num_train_steps)
        print("warmup start: ", warmup_start)
        scheduler = transformers.get_scheduler(cfg.scheduler, optimizer, warmup_start, num_train_steps)
        
        return scheduler
    
    model = FB3Model(cfg).to(device)
    criterion = nn.SmoothL1Loss(reduction = 'mean')
    optimizer = get_optimizer(cfg, model)
    scheduler = get_scheduler(cfg, optimizer, len(train_loader))

    best_score = np.inf
    best_val_loss = np.inf
    best_train_loss = np.inf

    for epoch in range(cfg.epochs):
        
        start = time.time()
        
        avg_loss = train_fn(model, criterion, optimizer, scheduler, train_loader, epoch, cfg)
        
        avg_val_loss, preds = valid_fn(model, criterion, valid_loader, epoch, cfg)
        score, scores = get_score(preds, valid_labels)
        
        elapsed = time.time() - start
        
        print(f"Epoch: {epoch+1} - avg_train_loss = {avg_loss:.4f} avg_val_loss: {avg_val_loss:.4f} time: {elapsed:.4f}s")
        print(f"Epoch: {epoch+1} - Score: {score:.4f}, Scores: {scores}")

        if best_score > score:
           best_score = score
           best_train_loss = avg_loss
           bset_val_loss = avg_val_loss
           
           torch.save({'model': model.state_dict(),
                       'predictions': preds},
                      CFG.OUTPUT_DIR + f"{CFG.model_name.replace('/', '-')}_fold{fold}_epoch{epoch+1}.pth")
    
    torch.cuda.empty_cache()
    gc.collect()

    return best_train_loss, best_val_loss, best_score

In [ ]:
for fold in range(CFG.n_folds):
    best_train_loss, best_val_loss, best_score = train_loop(df_train, fold, CFG)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.w

names:  ['.embeddings.']
names:  ['encoder.LayerNorm', 'rel_embeddings']
names:  ['.0.', '.1.', '.2.', '.3.', '.4.', '.5.']
names:  ['.6.', '.7.', '.8.', '.9.', '.10.', '.11.']
names:  ['.12.', '.13.', '.14.', '.15.', '.16.', '.17.', '.18.', '.19.', '.20.', '.21.', '.22.']
names:  ['.23.']
warmup start:  7825


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[0][19/1565] Elapsed: 0m 20s (remain 26m 59s) Loss: 2.6929(2.4295) Grad: 419478.875 LR: 0.00000077 
[0][39/1565] Elapsed: 0m 41s (remain 26m 23s) Loss: 2.3067(2.4609) Grad: 417741.875 LR: 0.00000153 
[0][59/1565] Elapsed: 1m 1s (remain 25m 53s) Loss: 2.1510(2.4950) Grad: 419321.65625 LR: 0.00000230 
[0][79/1565] Elapsed: 1m 22s (remain 25m 28s) Loss: 2.6312(2.4824) Grad: 415949.15625 LR: 0.00000307 
[0][99/1565] Elapsed: 1m 42s (remain 25m 6s) Loss: 2.5555(2.4301) Grad: 401013.4375 LR: 0.00000383 
[0][119/1565] Elapsed: 2m 3s (remain 24m 45s) Loss: 2.0924(2.4091) Grad: 415186.6875 LR: 0.00000460 
[0][139/1565] Elapsed: 2m 23s (remain 24m 23s) Loss: 2.8557(2.3796) Grad: 419205.03125 LR: 0.00000537 
[0][159/1565] Elapsed: 2m 44s (remain 24m 2s) Loss: 1.7566(2.3601) Grad: 414584.375 LR: 0.00000613 
[0][179/1565] Elapsed: 3m 4s (remain 23m 41s) Loss: 2.2624(2.3205) Grad: 435310.8125 LR: 0.00000690 
[0][199/1565] Elapsed: 3m 25s (remain 23m 20s) Loss: 1.8345(2.2596) Grad: 424898.3125 LR: 0.

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

[0][19/1563] Elapsed: 0m 20s (remain 26m 39s) Loss: 2.3902(2.6684) Grad: 417293.1875 LR: 0.00000077 
[0][39/1563] Elapsed: 0m 41s (remain 26m 18s) Loss: 2.2176(2.6371) Grad: 425452.375 LR: 0.00000154 
[0][59/1563] Elapsed: 1m 1s (remain 25m 52s) Loss: 2.3879(2.5949) Grad: 420452.40625 LR: 0.00000230 
[0][79/1563] Elapsed: 1m 22s (remain 25m 26s) Loss: 2.4751(2.5634) Grad: 415011.21875 LR: 0.00000307 
[0][99/1563] Elapsed: 1m 42s (remain 25m 4s) Loss: 2.8357(2.5895) Grad: 420136.0625 LR: 0.00000384 
[0][119/1563] Elapsed: 2m 3s (remain 24m 42s) Loss: 1.8722(2.5649) Grad: 420157.8125 LR: 0.00000461 
[0][139/1563] Elapsed: 2m 23s (remain 24m 21s) Loss: 2.1325(2.5393) Grad: 419238.34375 LR: 0.00000537 
[0][159/1563] Elapsed: 2m 44s (remain 24m 0s) Loss: 2.2890(2.5024) Grad: 426907.03125 LR: 0.00000614 
[0][179/1563] Elapsed: 3m 4s (remain 23m 38s) Loss: 3.2267(2.4874) Grad: 442663.75 LR: 0.00000691 
[0][199/1563] Elapsed: 3m 25s (remain 23m 18s) Loss: 2.0252(2.4412) Grad: 422285.21875 LR: 

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[1][19/1563] Elapsed: 0m 20s (remain 26m 40s) Loss: 0.1015(0.1312) Grad: 304177.4375 LR: 0.00006077 
[1][39/1563] Elapsed: 0m 41s (remain 26m 15s) Loss: 0.1321(0.1303) Grad: 282355.78125 LR: 0.00006154 
[1][59/1563] Elapsed: 1m 1s (remain 25m 51s) Loss: 0.1165(0.1239) Grad: 165624.734375 LR: 0.00006230 
[1][79/1563] Elapsed: 1m 22s (remain 25m 26s) Loss: 0.2344(0.1251) Grad: 500392.84375 LR: 0.00006307 
[1][99/1563] Elapsed: 1m 42s (remain 25m 4s) Loss: 0.1202(0.1224) Grad: 173147.0 LR: 0.00006384 
[1][119/1563] Elapsed: 2m 3s (remain 24m 43s) Loss: 0.0626(0.1236) Grad: 210051.03125 LR: 0.00006461 
[1][139/1563] Elapsed: 2m 23s (remain 24m 21s) Loss: 0.1253(0.1234) Grad: 289981.625 LR: 0.00006537 
[1][159/1563] Elapsed: 2m 44s (remain 24m 0s) Loss: 0.1231(0.1246) Grad: 331749.71875 LR: 0.00006614 
[1][179/1563] Elapsed: 3m 4s (remain 23m 39s) Loss: 0.1693(0.1241) Grad: 500883.0 LR: 0.00006691 
[1][199/1563] Elapsed: 3m 25s (remain 23m 18s) Loss: 0.1509(0.1227) Grad: 446165.0625 LR: 0.0

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:249: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[2][19/1563] Elapsed: 0m 20s (remain 26m 33s) Loss: 0.0805(0.1308) Grad: 244140.953125 LR: 0.00012077 
[2][39/1563] Elapsed: 0m 41s (remain 26m 5s) Loss: 0.1591(0.1211) Grad: 306919.90625 LR: 0.00012154 
[2][59/1563] Elapsed: 1m 1s (remain 25m 42s) Loss: 0.2723(0.1227) Grad: 621501.125 LR: 0.00012230 
[2][79/1563] Elapsed: 1m 22s (remain 25m 20s) Loss: 0.0687(0.1246) Grad: 137161.484375 LR: 0.00012307 
[2][99/1563] Elapsed: 1m 42s (remain 24m 59s) Loss: 0.0655(0.1241) Grad: 191670.859375 LR: 0.00012384 
[2][119/1563] Elapsed: 2m 2s (remain 24m 38s) Loss: 0.1266(0.1235) Grad: 352398.90625 LR: 0.00012461 
[2][139/1563] Elapsed: 2m 23s (remain 24m 17s) Loss: 0.0931(0.1215) Grad: 329270.84375 LR: 0.00012537 
[2][159/1563] Elapsed: 2m 43s (remain 23m 56s) Loss: 0.1328(0.1218) Grad: 432949.125 LR: 0.00012614 
[2][179/1563] Elapsed: 3m 4s (remain 23m 35s) Loss: 0.1233(0.1200) Grad: 204926.59375 LR: 0.00012691 
[2][199/1563] Elapsed: 3m 24s (remain 23m 15s) Loss: 0.0635(0.1182) Grad: 184977.57